In [1]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
### Libraries 
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../codes/')
from myUtils import *

Populating the interactive namespace from numpy and matplotlib


In [4]:
dfTV_day = pd.read_pickle('../data_revision/results/df_res_day_new.pkl')
dfTV_night = pd.read_pickle('../data_revision/results/df_res_night_new.pkl')
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')

In [18]:
############## Define Betas ##########################################
phases = [0,1,2,3,4]
betas_day = []
betas_night = []
for phase in phases:
    tmp = dfTV_day[dfTV_day['Phase']==phase].reset_index(drop=True)
    idx = np.where(tmp['R2_val_slm']==tmp['R2_val_slm'].max())[0][0]
    betas_day.append(tmp['SLM'][idx])
    
    tmp = dfTV_night[dfTV_night['Phase']==phase].reset_index(drop=True)
    idx = np.where(tmp['R2_val_slm']==tmp['R2_val_slm'].max())[0][0]
    betas_night.append(tmp['SLM'][idx])
    

In [19]:
betasDay = np.mean(betas_day, axis=0)
betasNight = np.mean(betas_night, axis=0)

In [22]:
######### Load datasets for cross validation
dfTV = pd.read_pickle('../data_revision/cities/cv/df_train_val.pkl')
dfTest = pd.read_pickle('../data_revision/cities/cv/df_test.pkl')

In [23]:
idx = np.arange(0,200)
gdf_all = pd.DataFrame()
for ii in idx:
    gdf = gpd.read_file('../data_revision/GEE_dataframes/gdf_%d.shp'%ii)
    gdf =  gdf.set_crs('epsg:4326')
    gdf = gdf.to_crs("ESRI:54009")
    gdf['dist'] = -gdf['dist']
    gdf = gdf[gdf['dist']>0]
    gdf['dist_n'] = 1/(gdf['dist']**2)
    gdf['dist_n'] = norm01(gdf, 'dist_n')
    gdf = gdf[gdf['NDVI']>=-1]
    gdf = gdf[gdf['NDBI']>=-1].reset_index(drop=True)
    gdf['hot_days']=gdf['hot_days']/11
    gdf['hot_nights']=gdf['hot_nights']/11
    gdf_all = gdf_all.append(gdf, ignore_index=True)

In [27]:
target_vars = ['hot_days', 'hot_nights']
predictor_vars = ['NDVI', 'dist_n']
target_var = target_vars[0]
cols = ['R2', 'betas']
dftest_day = pd.DataFrame(columns=cols)
phases = dfTest['phase'].unique()
for phase in phases:
    c = list(dfTest[dfTest['phase']==phase]['UC_NM_MN'])
    gdf_test = gdf_all[gdf_all['city'].isin(c)].reset_index(drop=True)
    w_test = libpysal.weights.KNN.from_dataframe(gdf_test, k=8)
    w_test.transform = 'r'
    w_test = libpysal.weights.lag_spatial(w_test, gdf_test[target_var])
    x_test = gdf_test[predictor_vars].values
    constant = sm.add_constant(np.hstack((x_test,np.array(w_test).reshape(-1,1))))
    ytest_  = np.sum(constant * (betasDay.T), axis=1).reshape((-1, 1))

    print(r2_score(ytest_, gdf_test[target_var]))
    data = [r2_score(ytest_, gdf_test[target_var]), betasDay.T]
    dftest_day = dftest_day.append(pd.DataFrame(columns=cols,data=[data]),ignore_index=True)

0.6257626397028182
0.5633620964090726
0.6224962812818711
0.5673660135376675
0.5896196704455511


In [28]:
predictor_vars = ['NDVI', 'dist_n']
target_var = target_vars[1]
cols = ['R2', 'betas']
dftest_night = pd.DataFrame(columns=cols)
phases = dfTest['phase'].unique()
for phase in phases:
    c = list(dfTest[dfTest['phase']==phase]['UC_NM_MN'])
    gdf_test = gdf_all[gdf_all['city'].isin(c)].reset_index(drop=True)
    w_test = libpysal.weights.KNN.from_dataframe(gdf_test, k=8)
    w_test.transform = 'r'
    w_test = libpysal.weights.lag_spatial(w_test, gdf_test[target_var])
    x_test = gdf_test[predictor_vars].values
    constant = sm.add_constant(np.hstack((x_test,np.array(w_test).reshape(-1,1))))
    ytest_  = np.sum(constant * (betasNight.T), axis=1).reshape((-1, 1))

    print(r2_score(ytest_, gdf_test[target_var]))
    data = [r2_score(ytest_, gdf_test[target_var]), betasNight.T]
    dftest_night =dftest_night.append(pd.DataFrame(columns=cols,data=[data]),ignore_index=True)

0.8894906496981672
0.864373667815349
0.872946277922001
0.8894827531307669
0.8652352175240821


In [31]:
dftest_day.to_pickle('../data_revision/results/dftest_day_new.pkl')
dftest_night.to_pickle('../data_revision/results/dftest_night_new.pkl')

In [33]:
betasDay.T

array([[ 3.64209653, -4.67311115, -1.16021593,  0.62546181]])

In [34]:
betasNight.T

array([[ 2.23640437, -2.68122998, -0.59718829,  0.79733414]])